In [247]:
from langchain.chains.combine_documents import create_stuff_documents_chain
import os
from dotenv import load_dotenv
import json
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, BasePromptTemplate  
import chromadb
from langchain.chains.combine_documents import create_stuff_documents_chain     
from langchain.chains import create_history_aware_retriever
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import create_retrieval_chain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.memory import ChatMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from chromadb import Documents, EmbeddingFunction, Embeddings
from langchain_chroma import Chroma
# Load environment variables from .env file
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")


In [202]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=gemini_api_key,convert_system_message_to_human=True)

In [185]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=gemini_api_key)

In [151]:
import google.generativeai as genai

genai.configure(api_key=gemini_api_key)

model = genai.GenerativeModel('gemini-1.5-flash')
translator_model = genai.GenerativeModel('gemini-1.5-pro')


In [166]:
class GeminiEmbeddingFunction(EmbeddingFunction):

    def embed_query():
      model = 'models/embedding-001'
      title = "Custom query"
      
      
      return genai.embed_content(model=model,
                                  content=input,
                                  task_type="retrieval_document",
                                  title=title)["embedding"]

In [49]:
def ready_prompt(user_query):

    prompt = f'''

    Prompt:
    You are a language model tasked with classifying user queries into one or more of the following categories:
    1.	Soil
    2.	Irrigation
    3.	Plant Diseases/Botany
    Each query can be associated with one or more of these labels. Your response should be formatted as a valid JSON object, specifying the number of labels assigned to the query and the labels themselves in the form of a Python list.
    Response Format:

    {{
        "RESPONSE": {{
            "answer": "<Number of labels assigned to the query>",
            "labels": <Labels according to you (should be a valid python list)>
        }}
    }}
    Example Query: "How can I improve soil fertility and what are the common diseases affecting tomatoes?"
    Example Response:
    {{
        "RESPONSE": {{
            "answer": "2",
            "labels": [1, 3]
        }}
    }}
    Your Task: For each user query, analyze the content and determine the appropriate labels (1 for Soil, 2 for Irrigation, and 3 for Plant Diseases/Botany). Provide the number of labels and the corresponding labels in the specified JSON format.
    Examples-

    Example 1: Single Label
    Query: "What type of soil is best for growing carrots?"
    Response:
    {{
        "RESPONSE": {{
            "answer": "1",
            "labels": [1]
        }}
    }}
    Example 2: Single Label
    Query: "How often should I water my cucumber plants?"
    Response:
    {{
        "RESPONSE": {{
            "answer": "1",
            "labels": [2]
        }}
    }}
    Example 3: Single Label
    Query: "What are the symptoms of powdery mildew on squash plants?"
    Response:
    {{
        "RESPONSE": {{
            "answer": "1",
            "labels": [3]
        }}
    }}
    
    Example 4: Multi-Label
    Query: "What soil amendments are needed to improve water retention, and how do I prevent root rot in tomatoes?"
    Response:
    {{
        "RESPONSE": {{
            "answer": "2",
            "labels": [1, 3]
        }}
    }}
    Example 5: Multi-Label
    Query: "How can I set up a drip irrigation system and what are the benefits for soil health?"
    Response:
    {{
        "RESPONSE": {{
            "answer": "2",
            "labels": [1, 2]
        }}
    }}
    Example 6: Multi-Label
    Query: "What are the best practices for watering plants to avoid fungal diseases?"
    Response:
    {{
        "RESPONSE": {{
            "answer": "2",
            "labels": [2, 3]
        }}
    }}

    User Query: {user_query}


    '''
    
    return prompt

In [50]:
def gemini_pro_with_for_generation_of_subproblems(complex_query):

    prompt = f'''

    You are tasked with breaking down a complex query into three specific categories: Soil, Irrigation, and Plant Diseases. A complex query consists of a single question that requires multiple operations to be performed simultaneously. For each category you determine to be relevant, rephrase the original user's question and break it into sub-problems. Ensure that the core meaning of the original user's question remains intact while dividing it into sub-problems.
    Your output should be in a well-defined valid JSON format as follows:
    {{
        "RESPONSE": {{
            "sub-problems": ["<list of sub-problems separated by commas as a Python list>"],
            "labels": ["<Python list of labels for each of the sub-problems devised>"]
        }}
    }}
    Examples:
    1.	Example Query: "How can I improve the soil quality, ensure proper irrigation, and prevent common diseases in my tomato plants?"
    {{
        "RESPONSE": {{
            "sub-problems": [
                "What are the best practices to improve soil quality for tomato plants?",
                "How can I ensure proper irrigation for tomato plants?",
                "What are the common diseases that affect tomato plants and how can I prevent them?"
            ],
            "labels": ["Soil", "Irrigation", "Plant Diseases"]
        }}
    }}
    2.	Example Query: "What are the necessary soil amendments, irrigation techniques, and disease control measures for growing healthy cucumbers?"
    {{
        "RESPONSE": {{
            "sub-problems": [
                "What soil amendments are necessary for growing healthy cucumbers?",
                "What irrigation techniques are best for cucumber plants?",
                "What disease control measures should I take for cucumber plants?"
            ],
            "labels": ["Soil", "Irrigation", "Plant Diseases"]
        }}
    }}
    3.	Example Query: "How do I prepare the soil and set up an efficient irrigation system for my new vineyard?"
    {{
        "RESPONSE": {{
            "sub-problems": [
                "How do I prepare the soil for a new vineyard?",
                "What steps should I take to set up an efficient irrigation system for a vineyard?"
            ],
            "labels": ["Soil", "Irrigation"]
        }}
    }}
    4.	Example Query: "What are the best soil treatments and how do I prevent diseases in my rose garden?"
    {{
        "RESPONSE": {{
            "sub-problems": [
                "What are the best soil treatments for a rose garden?",
                "How do I prevent diseases in a rose garden?"
            ],
            "labels": ["Soil", "Plant Diseases"]
        }}
    }}
    5.	Example Query: "Can you suggest irrigation methods and disease prevention tips for my apple orchard?"
    {{
        "RESPONSE": {{
            "sub-problems": [
                "What irrigation methods are suitable for an apple orchard?",
                "What are the best tips for preventing diseases in an apple orchard?"
            ],
            "labels": ["Irrigation", "Plant Diseases"]
        }}
    }}
    6.	Example Query: "How should I treat the soil and what irrigation system is ideal for my herb garden?"
    {{
        "RESPONSE": {{
            "sub-problems": [
                "How should I treat the soil for an herb garden?",
                "What irrigation system is ideal for an herb garden?"
            ],
            "labels": ["Soil", "Irrigation"]
        }}
    }}

    User query: {complex_query}
    '''
    
    res = model.generate_content(
        contents=prompt
    )
    return res.text


In [51]:
def gemini_pro_with_for_multi_label_classification(user_query):
    
    prompt = ready_prompt(user_query=user_query)
    
    res = model.generate_content(
        contents=prompt
    )
    return res.text


In [52]:
def check_simple_or_complexy_query(query):
    res = gemini_pro_with_for_multi_label_classification(query)
    res = res.replace('```', '').replace("\n", "").replace('json','')
    json_res = json.loads(res)
    if len(json_res['RESPONSE']['labels']) > 1:
        return "COMPLEX"
    else:
        return "SIMPLE"

In [53]:
def translate(query,src, tgt):
    prompt = f'''
    
    Trnslate this sentence {query} from {src} language to {tgt} language keeping the essence, syntactic and semantic meaning the same.
    '''
    res = model.generate_content(contents=prompt).text
    return res

In [188]:
query

'What kind of bookworms and catepillars appear due to climate change?'

In [191]:
# save to disk
import pickle
chunks = pickle.load(open('./RAG/chunks.pkl', 'rb'))
db2 = Chroma.from_texts(chunks, gemini_embeddings, persist_directory="./chroma_db")
docs = db2.similarity_search(query)


OperationalError: attempt to write a readonly database

In [ ]:
docs

In [54]:
def load_chromadb():
    # client = chromadb.PersistentClient(path="RAG/vectorSearchForFarmGenie")
    # chroma_db = client.get_collection(collection_name)
    # return chroma_db 
    # load from disk
    query = "What kind of bookworms and catepillars appear due to climate change?"
    db = Chroma(persist_directory="RAG/vectorSearchForFarmGenie", embedding_function=gemini_embeddings)
    docs = db.similarity_search(query)  
    print(docs)

In [55]:
def CreateEmbeddings(query):
    model = 'models/embedding-001'
    embedding = genai.embed_content(model=model,
                                content=query,
                                task_type="retrieval_query",
)
    return embedding

In [134]:
def rag_setup(query):
    
    collection_name = 'embeddings_for_farm_book'
    db = load_chromadb(collection_name=collection_name)
    query = "What kind of bookworms and catepillars appear due to climate change?"
    query_embeddings = CreateEmbeddings(query)
    query_embeddings = query_embeddings['embedding']
    # query_embeddings = create_embeddings(query)
    result = db.query(query_embeddings=query_embeddings, n_results=1).get('documents')[0][0]
    return result

In [265]:
def CreateSubproblems(query):
    res = gemini_pro_with_for_generation_of_subproblems(query)
    res = res.replace('```', '').replace("\n", "").replace('json','')
    json_res = json.loads(res)
    json_res = json_res['subproblems']
    return json_res

In [136]:
from langchain_core.runnables import Runnable
from langchain_core.runnables import RunnableLambda


In [273]:
# @chain
def HE_TranslatorRunnable(input):
    
    src="Hindi"
    tgt = "English"
    translated_text = translate(input, {src}, {tgt})
    return translated_text

In [274]:
# @chain
def EH_TranslatorRunnable(input):
    
    src="English"
    tgt = "Hindi"
    translated_text = translate(input, {src}, {tgt})
    return translated_text

In [275]:

# @chain
def RAGRunnable(input):
   
    rag_out = rag_setup(input)
    
    return rag_out

In [276]:
# @chain
def MoERunnable(query):

    subproblems = CreateSubproblems(query)
    
    return subproblems

In [277]:
# @chain
def ConditionalRunnable(translated_input):
    res = check_simple_or_complexy_query(translated_input)
    if res == 'SIMPLE':
        output = RAGRunnable(translated_input)
        
    elif res == 'COMPLEX':
        output = MoERunnable(translated_input)
    return output, translated_input

In [194]:
# load from disk
db = Chroma(persist_directory="./RAG/vectorSearchForFarmGenie", embedding_function=gemini_embeddings, collection_name="embeddings_for_farm_book")
docs = db.similarity_search(query, k=2)
docs

[Document(page_content='due to climate change • Due to increase in rainfall: Pests like bollworm, red hairy caterpillar and leaf spot diseases may increase.Due to increase in temperature: Suck-ing pests such as mites and leaf miner may in-crease. • Due to variation in rainfall and temperature: Pest and diseases of crops to be altered because of more enhanced pathogen and vector devel-opment, rapid pathogen transmission and in-creased host susceptibility. Sometimes a minor pest may become a major pest. • Agricultural biodiversity is also threatened by decreased rainfall and increased temperature, sea level rise and increased frequency and se- verity of drought, cyclone and flood. Quality of farm products such as fruits, vegetables, tea, coffee, aromatic and medicinal plants may be affected. Water • Demand for irrigation to increase with in-creased temperature and higher amount of evapo-transpiration. This may result in lower - ing of groundwater table at some places. • The melting of gl

In [278]:
# Initialize memory
# memory = ConversationBufferMemory()

chat_history = []

In [279]:
# @chain
def OutputRunnable(output):
    
    vectorstore = Chroma(persist_directory="./RAG/vectorSearchForFarmGenie", embedding_function=gemini_embeddings, collection_name="embeddings_for_farm_book")
    retriever = vectorstore.as_retriever(k=2)

    # chat_history = ChatMessageHistory()
    
    # chat_history.add_user_message(query)
    
    # # print(chat_history.messages)
    
    system_prompt = (
        "You are an assistant for question-answering tasks in a human-like conversational manner. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know."
        "\n\n"
        "{context}"
    
    )
    
    
    
    # # System prompt for answering
    # system_prompt = (
    #     "You are an assistant for question-answering tasks in a human-like conversational manner. "
    #     "Use the following pieces of retrieved context to answer "
    #     "the question. If you don't know the answer, say that you "
    #     "don't know."
    #     "\n\n"
    #     "{context}"
    # )

    # Create contextualize question prompt
    contextualize_q_system_prompt = (
        "Given a chat history and the latest user question "
        "which might reference context in the chat history, "
        "formulate a standalone question which can be understood "
        "without the chat history. Do NOT answer the question, "
        "just reformulate it if needed and otherwise return it as is."
    )
    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )

    # Create QA prompt
    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)



    # question = "What is Task Decomposition?"
    ai_msg_1 = rag_chain.invoke({"input": query, "chat_history": chat_history})
    # print(ai_msg_1['answer'])
    
    chat_history.extend(
        [
            HumanMessage(content=query),
            AIMessage(content=ai_msg_1["answer"]),
        ]
    )
    
    return ai_msg_1['answer']

    # second_question = "How to get rid of them?"
    # ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

    # print(ai_msg_2["answer"])



    # prompt = ChatPromptTemplate.from_messages(  
    # [
    #     ("system", system_prompt),
    #     ("human", "{input}"),
    # ]
    # )
    
    # document_chain = create_stuff_documents_chain(llm, prompt)
    # rag_chain = create_retrieval_chain(retriever, document_chain)
    # response = rag_chain.invoke({"input": query})
    # document_chain.invoke(
    #     {
    #         "messages": chat_history.messages,
    #         "context": output
    #     }
    # )
    

    
    # # Statefully manage chat history
    # store = {}

    # def get_session_history(session_id: str) -> BaseChatMessageHistory:
    #     if session_id not in store:
    #         store[session_id] = ChatMessageHistory()
    #     return store[session_id]

    # class RunnableWithMessageHistory(Runnable):
    #     def __init__(self, runnable, get_session_history, input_messages_key, history_messages_key, output_messages_key):
    #         self.runnable = runnable
    #         self.get_session_history = get_session_history
    #         self.input_messages_key = input_messages_key
    #         self.history_messages_key = history_messages_key
    #         self.output_messages_key = output_messages_key

    #     def invoke(self, input, config=None):
    #         session_id = config.get("configurable", {}).get("session_id")
    #         chat_history = self.get_session_history(session_id)
    #         chat_history.add_message({"role": "user", "content": input[self.input_messages_key]})

    #         augmented_input = {
    #             self.history_messages_key: chat_history.messages,
    #             **input
    #         }

    #         result = self.runnable.invoke(augmented_input, config=config)

    #         chat_history.add_message({"role": "assistant", "content": result[self.output_messages_key]})
    #         return result

    # conversational_rag_chain = RunnableWithMessageHistory(
    #     rag_chain,
    #     get_session_history,
    #     input_messages_key="input",
    #     history_messages_key="chat_history",
    #     output_messages_key="answer",
    # )

    # answer = conversational_rag_chain.invoke(
    #     {"input": query},
    #     config={"configurable": {"session_id": "abc123"}}
    # )["answer"]

    # return response




In [280]:
# query = "Hey wanna flirt?"
query = "कृषि में जलवायुीय परिवर्तन के दौरान किस प्रकार के कैटपिलर्स और कीट होते हैं?"
he_translator = RunnableLambda(HE_TranslatorRunnable)
eh_translator = RunnableLambda(EH_TranslatorRunnable)
condition = RunnableLambda(ConditionalRunnable)
output = RunnableLambda(OutputRunnable)

pipeline = he_translator | condition | output | eh_translator

# chain = ({
    
# })
result = pipeline.invoke({"query": query})
print(result)

मुझे माफ़ करना, लेकिन मैं इस सवाल का जवाब नहीं दे सकता हूँ कि नाशपाती और कीड़े जलवायु परिवर्तन के दौरान कृषि में कैसे प्रभावित होते हैं। दिया गया पाठ नागपुर मंडरिन के लिए कृषि पद्धतियों पर केंद्रित है, जिसमें खरपतवार नियंत्रण, फल गिरावट, मिट्टी और पौधे पोषण और कीट प्रबंधन शामिल है। इसमें जलवायु परिवर्तन के दौरान प्रचलित विशिष्ट नाशपाती या कीड़ों के बारे में जानकारी नहीं है। 



In [268]:
chat_history

[HumanMessage(content='What kinds of catepillars and pests are seen during climatic changes on agricultural lands/farms?'),
 AIMessage(content='The provided text mentions that due to an increase in rainfall, pests like bollworm, red hairy caterpillar and leaf spot diseases may increase. It also mentions that due to an increase in temperature, sucking pests such as mites and leaf miner may increase. \n'),
 HumanMessage(content='What did I just asked?'),
 AIMessage(content='You asked: "What kinds of caterpillars and pests are seen during climatic changes on agricultural lands/farms?" \n'),
 HumanMessage(content='Hey wanna flirt?'),
 AIMessage(content="I'm sorry, but I'm not programmed to flirt. I'm a helpful and harmless AI assistant.  Is there anything else I can help you with? \n"),
 HumanMessage(content='Hey wanna flirt?'),
 AIMessage(content="I understand you're trying to be playful, but as an AI, I don't have the capacity for flirting. My purpose is to provide helpful and harmless a

In [211]:
result

{'input': 'What kinds of catepillars and pests are seen during climatic changes on agricultural lands/farms?',
 'context': [Document(page_content='due to climate change • Due to increase in rainfall: Pests like bollworm, red hairy caterpillar and leaf spot diseases may increase.Due to increase in temperature: Suck-ing pests such as mites and leaf miner may in-crease. • Due to variation in rainfall and temperature: Pest and diseases of crops to be altered because of more enhanced pathogen and vector devel-opment, rapid pathogen transmission and in-creased host susceptibility. Sometimes a minor pest may become a major pest. • Agricultural biodiversity is also threatened by decreased rainfall and increased temperature, sea level rise and increased frequency and se- verity of drought, cyclone and flood. Quality of farm products such as fruits, vegetables, tea, coffee, aromatic and medicinal plants may be affected. Water • Demand for irrigation to increase with in-creased temperature and hi